# Miniproject 1
Fingermovements predicting

In [ ]:
# IMPORTS
import torch
from torch import Tensor, nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
import numpy as np
from math import log10

# Our own code
import helpers as HL
import models as ML

%reload_ext autoreload
%autoreload 2

#### Importing data

In [ ]:
train_data, test_data = HL.import_data(flatten=True)

### Create new models if you want

### Choose model and define loss criterion and train

In [ ]:
input_dim = len(train_data[0][0])
print(input_dim)
output_dim = 1
nb_hidden_layers = 2
hidden_width = 300
# initialize the model
densemodel_1 = ML.DenseNet(input_dim, output_dim, nb_hidden_layers=2)
linear_model = ML.Linear_regression_model(input_dim, output_dim)

model = densemodel_1
# define loss criterion
criterion = nn.MSELoss()


### Training model

In [ ]:
trained_model = HL.train_model(train_data, test_data, model, criterion, learning_rate=0.00005, epochs=100, batch_size=16)